# Strands Agents Fundamentals - Part 1

<div style="background-color: #FFE4B5; padding: 15px; border-radius: 5px; margin: 10px 0;width: 85%;">
  <strong>!! Important Note !!</strong><br>
  In this notebook we will learn some basic features of strands-agents. If you are already familiar with the foundations, please skip this notebook and continue with the rest of the labs. Most of the examples referrenced in this notebook are from <b><a href="https://strandsagents.com/">strands-agents official documentation</a></b>
</div>

[Strands Agents](https://strandsagents.com/latest/) is a simple-to-use, code-first framework for building agents.

- [strands-agents-github-repository](https://github.com/strands-agents)


## Pre-requisites

<div style="background-color: #FFE4B5; padding: 15px; width: 85%;"> <h4> <b> !! Important !! </b> </h4>
<b>Enable model access</b> through Amazon Bedrock Console and select the models you like to experiment today. Region <b>us-west-2</b>.<br/>
 
The default model provider used by strandsagents is [Amazon Bedrock](https://strandsagents.com/latest/user-guide/concepts/model-providers/amazon-bedrock) and the default model is <b>Claude 3.7 Sonnet in the US Oregon (us-west-2) region </b>.
Different model providers can be configured for agents by following the [quickstart guide](https://strandsagents.com/latest/user-guide/quickstart/#model-providers). </div>

In [ ]:
!pip install --upgrade pip 

### Install necessary strands-agents and mcp packages Agents SDK

The Strands Agents SDK also offers the [strands-agents-tools](https://pypi.org/project/strands-agents-tools/) - [GitHub](https://github.com/strands-agents/tools) and [strands-agents-builder](https://pypi.org/project/strands-agents-builder/) - [GitHub](https://github.com/strands-agents/agent-builder) package helps you build your own strands-agents and tools. We are not using agent-builder in todays workshop, so not installing the package.

In [ ]:
!pip install strands-agents strands-agents-tools mcp

## Section 1: Let's learn some basic strands-agents features

<div style="background-color: #FFE4B5; padding: 15px; border-radius: 5px; margin: 10px 0; font-size:13px; width:85%">
<strong>Note:</strong> If you like to run this code through <b>terminal</b> or <b>an IDE</b>, use the python file "/agenticai-for-retail/00-Introduction/strands-agents-fundamentals.py" <br/>
    <b>example:</b> <code>python -m strands-agents-fundamentals</code> <br/>
    Please make sure the pre-requisite packages are installed and a python virtual environment is setup. Refer the <a href="https://strandsagents.com/latest/user-guide/quickstart/"> <b>quickstart guide</b> </a>
</div>

### 1.a Simple Agent with default model parameters
Make sure you have access to the default model (refer pre-requisites section, for more details on supported model providers and necessary configurations)

In [ ]:
from strands import Agent

# Create an agent with default settings
agent = Agent()
print(agent.model.config)
# Ask the agent a question. Review the response and agent traces with token count & latency, etc.
agent("Tell me about agentic AI")

### 1.b Agent with a Bedrock model that is different than default model. Also, including a simple system prompt

In [ ]:
# Create an agent with a specific model by passing the model ID string - use the model you have enabled access for.

# agent = Agent(model="us.amazon.nova-premier-v1:0") # simple agent with specific model. alternate with system prompt in the next step.

agent = Agent(model="us.amazon.nova-premier-v1:0", system_prompt="You are a helpful assistant that provides concise answers. ")
agent("Tell me about agentic AI")

### 1.c Agent with custom model parameters and optionally custom boto3 configurations (example - readtimeout, retries, etc.)

The [BedrockModel](https://strandsagents.com/latest/api-reference/models/#strands.models.bedrock) class in Strands enables seamless integration with Amazon Bedrock's API, supporting:
- Text generation
- Multi-Modal understanding (Image, Document, etc.)
- Tool/function calling
- Guardrail configurations
- System Prompt, Tool, and/or Message caching

We will experiment with some of these features. Feel free to refer above documentation and try out other supported features as well

In [ ]:
from strands.models import BedrockModel
from botocore.config import Config as BotocoreConfig

# Create a boto client config with custom settings
boto_config = BotocoreConfig(
    retries={"max_attempts": 2, "mode": "standard"},
    connect_timeout=5,
    read_timeout=30
)

# Create a Bedrock model instance
bedrock_model = BedrockModel(
    model_id="us.amazon.nova-premier-v1:0", # try with different models you enabled access
    region_name="us-west-2",
    temperature=0.3,
    top_p=0.8,
    boto_client_config=boto_config,
)

# Create an agent using the BedrockModel instance
agent = Agent(model=bedrock_model)

# Use the agent
agent_response_with_nova_premier = agent("Tell me about Amazon Bedrock.")

In [ ]:
# storing the object in local memory to use in Level-2 notebook
%store agent_response_with_nova_premier

### 1.d Agent with debug logs enabled

Strands SDK uses Python's standard logging module to provide visibility into its operations.

#### Log Levels:
- **DEBUG:** Used throughout the SDK for detailed operational information, particularly for tool registration, discovery, configuration, and execution flows.
- **WARNING:** Used to indicate potential issues that don't prevent operation, such as tool validation failures, specification validation errors, and context window overflow conditions.
- **ERROR:** Used to report significant problems that prevent specific operations from completing successfully, such as tool execution failures, event loop cycle exceptions, and handler errors.


In [ ]:
import logging

# Enables Strands debug log level
logging.getLogger("strands").setLevel(logging.DEBUG)

# Sets the logging format and streams logs. prints the detailed trace steps and event loop cycles
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)

agent = Agent() # using default model

agent("Tell me about Amazon Bedrock.")

Disabling DEBUG log for better readability in the next steps. DEBUG log might print quite a few logs in the subsequent steps, especillay when you use tools

In [ ]:
# Disable Strands debug log level
logging.getLogger("strands").setLevel(logging.ERROR)

### **1.e Tools**

Tools are the primary mechanism for extending agent capabilities, enabling them to perform actions beyond simple text generation. Tools allow agents to interact with external systems, access data, and manipulate their environment.
Strands offers built-in example tools to get started quickly experimenting with agents and tools during development. 
For more information, see [Example Built-in Tools](https://strandsagents.com/latest/user-guide/concepts/tools/example-tools-package/). The package is also open source and available on [GitHub](https://github.com/strands-agents/tools).

You can also define [python-based tools](https://strandsagents.com/latest/user-guide/concepts/tools/python-tools/) in Strands. This can be done either using [@tool](https://strandsagents.com/latest/api-reference/tools/#strands.tools.decorator.tool) decorator ir by defining a tool specification.

**Below example uses default example strands-tools ([calculator](https://github.com/strands-agents/tools/blob/main/src/strands_tools/calculator.py), [current_time](https://github.com/strands-agents/tools/blob/main/src/strands_tools/calculator.py), [python_repl](https://github.com/strands-agents/tools/blob/main/src/strands_tools/python_repl.py) ) and a custom tool (*letter_counter*) defined using a python decorator.**

In [ ]:
from strands import tool
from strands_tools import calculator, current_time, generate_image

from PIL import Image
import matplotlib.pyplot as plt

import pathlib
from pathlib import Path


# Define a custom tool as a Python function using the @tool decorator
@tool
def letter_counter(word: str, letter: str) -> int:
    """
    Count occurrences of a specific letter in a word.

    Args:
        word (str): The input word to search in
        letter (str): The specific letter to count

    Returns:
        int: The number of occurrences of the letter in the word
    """
    if not isinstance(word, str) or not isinstance(letter, str):
        return 0

    if len(letter) != 1:
        raise ValueError("The 'letter' parameter must be a single character")

    return word.lower().count(letter.lower())

# [OPTIONAL STATEMENT] 
# Set the logging level to ERROR if the agent response is not readable with the DEBUG logs. 
# please switch back to DEBUG if you want to look at the detailed traces of the agent processing steps.

# logging.getLogger("strands").setLevel(logging.ERROR) # uncomment this statement to disable DEBUG logs and enable ERROR logs only.

# Create an agent with tools from the strands-tools example tools package
# as well as our custom letter_counter tool
agent = Agent(tools=[calculator, current_time, generate_image, letter_counter])


# Ask the agent a question that uses the available tools
message = """
I have 4 requests:

1. What is the time right now?
2. Calculate 3111696 / 74088
3. Tell me how many letter R's are in the word "strawberry" 🍓
4. Generate an image of a futuristic city with flying cars
"""

tools_agent_response = agent(message)


## display the generated image. if you notice errors with the image display, check the image path ##

#Get image paths
folder_path = "output"
folder = Path(folder_path)
image_extensions = {'.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff', '.webp'}

for image_path in folder.iterdir():
    if image_path.is_file() and image_path.suffix.lower() in image_extensions:
        # Load and display
        print (image_path)
        img = Image.open(image_path)
        plt.imshow(img)
        plt.axis('off')
        plt.show()

### **1.f Observability**

In the Strands Agents SDK, observability refers to the ability to measure system behavior and performance. [Observability](https://strandsagents.com/latest/user-guide/observability-evaluation/observability/) is the combination of instrumentation, data collection, and analysis techniques that provide insights into an agent's behavior and performance.
Building observable agents starts with monitoring the right telemetry. While we leverage the same fundamental building blocks as traditional software — [traces](https://strandsagents.com/latest/user-guide/observability-evaluation/traces/), [metrics](https://strandsagents.com/latest/user-guide/observability-evaluation/metrics/), and [logs](https://strandsagents.com/latest/user-guide/observability-evaluation/logs/) — their application to agents requires special consideration. We need to capture not only standard application telemetry but also AI-specific signals like model interactions, reasoning steps, and tool usage.

### [Metrics](https://strandsagents.com/latest/user-guide/observability-evaluation/metrics/)

**Metrics for the agent invocation using non-default model (Metrics for Step 1.c)**

In [ ]:
import pathlib
from pathlib import Path

In [ ]:
agent_response_with_nova_premier.metrics.get_summary()

#### **Metrics for the agent invocation using strands tools (Metrics for Step 1.e)**

In [ ]:
folder_path = "./output"
folder = Path(folder_path)
image_extensions = {'.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff', '.webp'}

for image_path in folder.iterdir():
    if image_path.is_file() and image_path.suffix.lower() in image_extensions:
        # Load and display
        print (image_path)
        img = Image.open(image_path)
        plt.imshow(img)
        plt.axis('off')
        plt.show()

In [ ]:
tools_agent_response.metrics.get_summary()

## Next Step

Congrats. You have completed Level 1 of the lab. Now open `02-Strands-Agents-Foundations-Level-2.ipynb` and follow the instructions to continue.